### Script -- for feature set - "e" (7 feature) ---> NRI calculation
##### (for comapring models) --- performance on testing dataset 
#### comparing model - clinical only v/s promoters only
#### comparing model - clinical only v/s all features
#### comparing model - promoter only v/s all features 

In [1]:
import sys
print(sys.version)

3.12.2 | packaged by conda-forge | (main, Feb 16 2024, 20:50:58) [GCC 12.3.0]


In [3]:
import sys
random_state = int(sys.argv[1])

In [12]:
#importing librariers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sksurv.linear_model import CoxnetSurvivalAnalysis
from sksurv.util import Surv
from sklearn.preprocessing import StandardScaler
import warnings
from sklearn.exceptions import FitFailedWarning
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", FitFailedWarning)
from sksurv.linear_model import CoxnetSurvivalAnalysis
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sksurv.util import Surv
from sksurv.ensemble import RandomSurvivalForest


In [13]:
#reading the all dataset---
input_all = pd.read_csv("Input.csv")

In [14]:
train_df, val_df = train_test_split(input_all, test_size=0.30, stratify=input_all['mRNA_Subtype'], random_state=random_state)

In [16]:
#now defining feature set for model -01 & model -2
feature_sets = {
    "clinical_only": ['Ki67', 'Size_cm','CNA_Subtype_Chr8p21_del', 'CNA_Subtype_Chr13q34_amp', 'CNA_Subtype_Chr20q13_amp'],
    "promoter_only": ['pr3004_huwe1', 'pr9001_cald1'],
    "all_features": ['Ki67', 'Size_cm','CNA_Subtype_Chr8p21_del',
                               'CNA_Subtype_Chr13q34_amp', 'CNA_Subtype_Chr20q13_amp','pr3004_huwe1', 'pr9001_cald1']
}

model_comparisons = [
    ("clinical_only", "all_features")
]

for model1_name, model2_name in model_comparisons:
    print(f"\n🔁 Running comparison: {model1_name} vs {model2_name}")
    
    # Get feature sets
    features_1 = feature_sets[model1_name]
    features_2 = feature_sets[model2_name]
    print(features_1)
    print(features_2)

    X = train_df[features_1]
    y = train_df[['RFS_Status', 'RFS_time_Months']]
    y_surv = Surv.from_dataframe('RFS_Status', 'RFS_time_Months', y) #false: 0, true: 1)

    #1) LASSO ------------------hyperparameter -- alpha
    coxnet_pipe_lasso = make_pipeline( CoxnetSurvivalAnalysis(l1_ratio = 1.0, alpha_min_ratio = 0.01, max_iter=100))
    #2) RIDGE ------------------hyperparamters -- alpha
    coxnet_pipe_ridge = make_pipeline( CoxnetSurvivalAnalysis(l1_ratio=1e-5, alpha_min_ratio = 0.01, max_iter=100))  ### range (0.0, 1.0] notice the round bracket
    #3) ELASTICNET -------------hyperparameters -- L1 and alpha
    coxnet_pipe_elastic = make_pipeline(CoxnetSurvivalAnalysis(max_iter=100, alpha_min_ratio= 0.01))


    #defining random survival forest plot -------------hyperparameters ----n_estimators, min_samples_split, min_samples_leaf
    ##search for best hyperparameters for random survival forest
    
    param_grid_rf = {
        'n_estimators': [100, 500,1000],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [5, 10, 15]
    }
    event_occurrences = y_surv['RFS_Status']
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)
    
    grid_search_rf = GridSearchCV(
        estimator=RandomSurvivalForest(random_state=random_state, n_jobs=1),
        param_grid=param_grid_rf,
        cv=cv.split(X, event_occurrences),
        n_jobs=1,
        error_score=0.5,
        verbose=1
    ).fit(X, y_surv)
    print("Best Hyperparameters:", grid_search_rf.best_params_)
    best_model = grid_search_rf.best_estimator_
    
    # Extract best hyperparameters
    min_samples_leaf = grid_search_rf.best_params_['min_samples_leaf']
    min_samples_split = grid_search_rf.best_params_['min_samples_split']
    n_estimators = grid_search_rf.best_params_['n_estimators']
    print(min_samples_leaf)
    print(min_samples_split)
    print(n_estimators)

    #search for best l1 for elastic net
    # Set up the parameter grid
    param_grid_l1_elastic = {'coxnetsurvivalanalysis__l1_ratio': [0.2, 0.3, 0.4,0.5, 0.6, 0.7, 0.8, 0.9]}
    
    # Perform grid search with cross-validation
    event_occurrences = y_surv['RFS_Status']
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=random_state)
    
    GSV_elastic_L1 = GridSearchCV(
        coxnet_pipe_elastic,
        param_grid_l1_elastic,
        cv=cv.split(X, event_occurrences),
        error_score=0.5,
        n_jobs=1,
    ).fit(X, y_surv)
    
    
    # Best parameters
    best_l1_ratio = GSV_elastic_L1.best_params_['coxnetsurvivalanalysis__l1_ratio']
    print(f"Best L1 Ratio: {best_l1_ratio}")

    #fit elastic model with best l1 ratio:
    coxnet_pipe_elastic = make_pipeline( CoxnetSurvivalAnalysis(l1_ratio=best_l1_ratio, alpha_min_ratio = 0.01, max_iter=100))  ### range (0.0, 1.0] notice the round bracket
    coxnet_pipe_lasso.fit(X, y_surv)
    coxnet_pipe_ridge.fit(X, y_surv)
    coxnet_pipe_elastic.fit(X, y_surv)

    #now i have to search for best alpha for each model
    estimated_alphas_lasso = coxnet_pipe_lasso.named_steps['coxnetsurvivalanalysis'].alphas_
    estimated_alphas_ridge = coxnet_pipe_ridge.named_steps['coxnetsurvivalanalysis'].alphas_
    estimated_alphas_elastic = coxnet_pipe_elastic.named_steps['coxnetsurvivalanalysis'].alphas_

    #now the training dataset is divided into 3-folds (stratified based on number of events in each class), and for that perform grid search
    event_occurrences = y_surv['RFS_Status']
    
    
    #cv = KFold(n_splits=5, shuffle=True, random_state=0)
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=random_state)
    
    GSV_alpha_lasso = GridSearchCV(
        coxnet_pipe_lasso,
        param_grid={"coxnetsurvivalanalysis__alphas": [[v] for v in estimated_alphas_lasso]},
        cv=cv.split(X, event_occurrences),
        error_score=0.5,
        n_jobs=1,
    ).fit(X, y_surv)
    
    GSV_alpha_ridge = GridSearchCV(
        coxnet_pipe_ridge,
        param_grid={"coxnetsurvivalanalysis__alphas": [[v] for v in estimated_alphas_ridge]},
        cv=cv.split(X, event_occurrences),
        error_score=0.5,
        n_jobs=1,
    ).fit(X, y_surv)
    
    GSV_alpha_elastic = GridSearchCV(
        coxnet_pipe_elastic,
        param_grid={"coxnetsurvivalanalysis__alphas": [[v] for v in estimated_alphas_elastic]},
        cv=cv.split(X, event_occurrences),
        error_score=0.5,
        n_jobs=1,
    ).fit(X, y_surv)


    alpha_lasso = GSV_alpha_lasso.best_params_["coxnetsurvivalanalysis__alphas"]
    alpha_ridge = GSV_alpha_ridge.best_params_["coxnetsurvivalanalysis__alphas"]
    alpha_elastic = GSV_alpha_elastic.best_params_["coxnetsurvivalanalysis__alphas"]
    
    
    print("alpha_lasso = ",alpha_lasso)
    print("alpha_ridge = ",alpha_ridge)
    print("alpha_elastic = ",alpha_elastic)

    #now constructing the model, taking all the samples
    model_lasso = CoxnetSurvivalAnalysis(alphas=alpha_lasso, fit_baseline_model=True, l1_ratio=1.0).fit(X, y_surv)
    model_ridge = CoxnetSurvivalAnalysis(alphas=alpha_ridge, fit_baseline_model=True, l1_ratio=1e-5).fit(X, y_surv)
    model_elastic = CoxnetSurvivalAnalysis(alphas=alpha_elastic, fit_baseline_model=True, l1_ratio=best_l1_ratio).fit(X, y_surv)
    model_rf = RandomSurvivalForest(n_estimators=n_estimators, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf, n_jobs=1, random_state=random_state).fit(X, y_surv)

    X_val = val_df[features_1]
    y_val = val_df[['RFS_Status', 'RFS_time_Months']]
    y_val_surv = Surv.from_dataframe('RFS_Status', 'RFS_time_Months', y_val) #false: 0, true: 1)

    #now running for model 2 
    X2= train_df[features_2]
    y = train_df[['RFS_Status', 'RFS_time_Months']]
    y_surv = Surv.from_dataframe('RFS_Status', 'RFS_time_Months', y) #false: 0, true: 1)
    #1) LASSO ------------------hyperparameter -- alpha
    coxnet_pipe_lasso2 = make_pipeline( CoxnetSurvivalAnalysis(l1_ratio = 1.0, alpha_min_ratio = 0.01, max_iter=100))
    #2) RIDGE ------------------hyperparamters -- alpha
    coxnet_pipe_ridge2 = make_pipeline( CoxnetSurvivalAnalysis(l1_ratio=1e-5, alpha_min_ratio = 0.01, max_iter=100))  ### range (0.0, 1.0] notice the round bracket
    #3) ELASTICNET -------------hyperparameters -- L1 and alpha
    coxnet_pipe_elastic2 = make_pipeline(CoxnetSurvivalAnalysis(max_iter=100, alpha_min_ratio= 0.01))

    param_grid_rf = {
        'n_estimators': [100, 500,1000],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [5, 10, 15]
    }
    event_occurrences = y_surv['RFS_Status']
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)
    
    grid_search_rf = GridSearchCV(
        estimator=RandomSurvivalForest(random_state=random_state, n_jobs=1),
        param_grid=param_grid_rf,
        cv=cv.split(X2, event_occurrences),
        n_jobs=1,
        error_score=0.5,
        verbose=1
    ).fit(X2, y_surv)
    print("Best Hyperparameters:", grid_search_rf.best_params_)
    best_model = grid_search_rf.best_estimator_
    
    # Extract best hyperparameters
    min_samples_leaf = grid_search_rf.best_params_['min_samples_leaf']
    min_samples_split = grid_search_rf.best_params_['min_samples_split']
    n_estimators = grid_search_rf.best_params_['n_estimators']
    print(min_samples_leaf)
    print(min_samples_split)
    print(n_estimators)

    #search for best l1 for elastic net
    # Set up the parameter grid
    param_grid_l1_elastic = {'coxnetsurvivalanalysis__l1_ratio': [0.2, 0.3, 0.4,0.5, 0.6, 0.7, 0.8, 0.9]}
    
    # Perform grid search with cross-validation
    event_occurrences = y_surv['RFS_Status']
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=random_state)
    
    GSV_elastic_L1 = GridSearchCV(
        coxnet_pipe_elastic2,
        param_grid_l1_elastic,
        cv=cv.split(X2, event_occurrences),
        error_score=0.5,
        n_jobs=1,
    ).fit(X2, y_surv)
    
    
    # Best parameters
    best_l1_ratio = GSV_elastic_L1.best_params_['coxnetsurvivalanalysis__l1_ratio']
    print(f"Best L1 Ratio: {best_l1_ratio}")

    #fit elastic model with best l1 ratio:
    coxnet_pipe_elastic2 = make_pipeline( CoxnetSurvivalAnalysis(l1_ratio=best_l1_ratio, alpha_min_ratio = 0.01, max_iter=100))  ### range (0.0, 1.0] notice the round bracket
    coxnet_pipe_lasso2.fit(X2, y_surv)
    coxnet_pipe_ridge2.fit(X2, y_surv)
    coxnet_pipe_elastic2.fit(X2, y_surv)

    #now i have to search for best alpha for each model
    estimated_alphas_lasso = coxnet_pipe_lasso2.named_steps['coxnetsurvivalanalysis'].alphas_
    estimated_alphas_ridge = coxnet_pipe_ridge2.named_steps['coxnetsurvivalanalysis'].alphas_
    estimated_alphas_elastic = coxnet_pipe_elastic2.named_steps['coxnetsurvivalanalysis'].alphas_


    #now the training dataset is divided into 3-folds (stratified based on number of events in each class), and for that perform grid search
    event_occurrences = y_surv['RFS_Status']
    
    from sklearn.model_selection import KFold
    from sklearn.model_selection import StratifiedKFold
    from sksurv.util import Surv
    #cv = KFold(n_splits=5, shuffle=True, random_state=0)
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=random_state)
    
    GSV_alpha_lasso = GridSearchCV(
        coxnet_pipe_lasso2,
        param_grid={"coxnetsurvivalanalysis__alphas": [[v] for v in estimated_alphas_lasso]},
        cv=cv.split(X2, event_occurrences),
        error_score=0.5,
        n_jobs=1,
    ).fit(X2, y_surv)
    
    GSV_alpha_ridge = GridSearchCV(
        coxnet_pipe_ridge2,
        param_grid={"coxnetsurvivalanalysis__alphas": [[v] for v in estimated_alphas_ridge]},
        cv=cv.split(X2, event_occurrences),
        error_score=0.5,
        n_jobs=1,
    ).fit(X2, y_surv)
    
    GSV_alpha_elastic = GridSearchCV(
        coxnet_pipe_elastic2,
        param_grid={"coxnetsurvivalanalysis__alphas": [[v] for v in estimated_alphas_elastic]},
        cv=cv.split(X2, event_occurrences),
        error_score=0.5,
        n_jobs=1,
    ).fit(X2, y_surv)

    alpha_lasso = GSV_alpha_lasso.best_params_["coxnetsurvivalanalysis__alphas"]
    alpha_ridge = GSV_alpha_ridge.best_params_["coxnetsurvivalanalysis__alphas"]
    alpha_elastic = GSV_alpha_elastic.best_params_["coxnetsurvivalanalysis__alphas"]
    
    
    print("alpha_lasso = ",alpha_lasso)
    print("alpha_ridge = ",alpha_ridge)
    print("alpha_elastic = ",alpha_elastic)

    #now constructing the model, taking all the samples
    model_lasso_final = CoxnetSurvivalAnalysis(alphas=alpha_lasso, fit_baseline_model=True, l1_ratio=1.0).fit(X2, y_surv)
    model_ridge_final = CoxnetSurvivalAnalysis(alphas=alpha_ridge, fit_baseline_model=True, l1_ratio=1e-5).fit(X2, y_surv)
    model_elastic_final = CoxnetSurvivalAnalysis(alphas=alpha_elastic, fit_baseline_model=True, l1_ratio=best_l1_ratio).fit(X2, y_surv)
    model_rf_final = RandomSurvivalForest(n_estimators=n_estimators, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf, n_jobs=1, random_state=random_state).fit(X2, y_surv)

    def check_cat(prob, thresholds):    #the following funstion defines the risk categories 
        cat = 0
        for i, v in enumerate(thresholds):
            if prob > v:
                cat = i + 1
        return cat

    #Build a cross‑classification matrix showing, for a subset of patients, how often they move
    #from one risk category under the reference model to another category under the new model.
    def make_cat_matrix(ref, new, indices, thresholds):
        num_cats = len(thresholds) + 1
        mat = np.zeros((num_cats, num_cats))
        for i in indices:
            row, col = check_cat(ref[i], thresholds), check_cat(new[i], thresholds)
            mat[row, col] += 1
        return mat

    def calculate_nri_at_time(y_val, ref_probs, new_probs, time_point, thresholds):
        # Identify individuals at risk at this time point
        mask = y_val["RFS_time_Months"] >= time_point
        y_filtered = y_val[mask]
        indices = y_filtered.index.tolist()
    
        if len(indices) == 0:
            return {"time": time_point, "nri_event": np.nan, "nri_nonevent": np.nan, "nri_total": np.nan}
    
        y_truth = y_filtered["RFS_Status"].values
        ref = ref_probs[indices]
        new = new_probs[indices]
    
        event_index = [i for i in range(len(y_truth)) if y_truth[i] == 1]
        nonevent_index = [i for i in range(len(y_truth)) if y_truth[i] == 0]
    
        event_mat = make_cat_matrix(ref, new, event_index, thresholds)
        nonevent_mat = make_cat_matrix(ref, new, nonevent_index, thresholds)
    
        # upward = more aggressive classification (toward high risk)
        events_up = np.triu(event_mat, k=1).sum()
        events_down = np.tril(event_mat, k=-1).sum()
    
        nonevents_up = np.triu(nonevent_mat, k=1).sum()
        nonevents_down = np.tril(nonevent_mat, k=-1).sum()
    
        n_events = len(event_index)
        n_nonevents = len(nonevent_index)
    
        nri_events = (events_up - events_down) / n_events if n_events > 0 else np.nan
        nri_nonevents = (nonevents_down - nonevents_up) / n_nonevents if n_nonevents > 0 else np.nan
    
        return {
            "time": time_point,
            "nri_event": nri_events,
            "nri_nonevent": nri_nonevents,
            "nri_total": nri_events + nri_nonevents if not np.isnan(nri_events + nri_nonevents) else np.nan
        }

    def compute_nri_for_survival_models(model_ref, model_new, X_val_base, X_val, y_val, time_points, thresholds):
        # ✅ Reset index to align with NumPy arrays
        y_val = y_val.reset_index(drop=True)
        
        # Predict survival functions
        surv_funcs_ref = model_ref.predict_survival_function(X_val_base)
        surv_funcs_new = model_new.predict_survival_function(X_val)
    
        # Convert to risk probabilities (1 - S(t)) at each time point
        def get_risk_probs(surv_funcs, times):
            return np.asarray([[1 - fn(t) for t in times] for fn in surv_funcs])  # shape (n_samples, len(times))
    
        risk_probs_ref = get_risk_probs(surv_funcs_ref, time_points)
        risk_probs_new = get_risk_probs(surv_funcs_new, time_points)
    
        results = []
        for i, t in enumerate(time_points):
            result = calculate_nri_at_time(
                y_val,
                ref_probs=risk_probs_ref[:, i],
                new_probs=risk_probs_new[:, i],
                time_point=t,
                thresholds=thresholds
            )
            results.append(result)
    
        return pd.DataFrame(results)

    X_val2 = val_df[features_2]
    time_points = np.percentile(y_val['RFS_time_Months'], [25, 50, 75])
    percentiles = [25, 50, 75]
    # Define risk thresholds for category classification
    risk_thresholds = [0.02, 0.1, 0.5, 0.95]
    # Run NRI computation ---- fo rf model 
    nri_df = compute_nri_for_survival_models(model_rf, model_rf_final, X_val, X_val2, y_val, time_points, risk_thresholds)
    nri_df["percentile"] = percentiles
    nri_df["random_state"] = random_state
    nri_df["model"] = "random_forest"
    nri_df["comaprison"] = f"\n🔁 Running comparison: {model1_name} vs {model2_name}"
    output_csv = "./7FEATURE_test.csv"

    # Save or append to CSV
    try:
        existing = pd.read_csv(output_csv)
        combined = pd.concat([existing, nri_df], ignore_index=True)
        combined.to_csv(output_csv, index=False)
    except FileNotFoundError:
        nri_df.to_csv(output_csv, index=False)

    lasso_df = compute_nri_for_survival_models(model_lasso, model_lasso_final, X_val, X_val2, y_val, time_points, risk_thresholds)
    lasso_df["percentile"] = percentiles
    lasso_df["random_state"] = random_state
    lasso_df["model"] = "LASSO"
    lasso_df["comaprison"] = f"\n🔁 Running comparison: {model1_name} vs {model2_name}"
    lasso_csv =  "./7FEATURE_test.csv"
    # Save or append to CSV
    try:
        existing = pd.read_csv(lasso_csv)
        combined = pd.concat([existing, lasso_df], ignore_index=True)
        combined.to_csv(lasso_csv, index=False)
    except FileNotFoundError:
        lasso_df.to_csv(lasso_csv, index=False)

    # Run NRI computation ---- fo rf model 
    ridge_df = compute_nri_for_survival_models(model_ridge, model_ridge_final, X_val, X_val2, y_val, time_points, risk_thresholds)
    ridge_df["percentile"] = percentiles
    ridge_df["random_state"] = random_state
    ridge_df["model"] = "RIDGE"
    ridge_df["comaprison"] = f"\n🔁 Running comparison: {model1_name} vs {model2_name}"

    # Set file to store cumulative NRI results
    ridge_csv =   "./7FEATURE_test.csv"
    
    # Save or append to CSV
    try:
        existing = pd.read_csv(ridge_csv)
        combined = pd.concat([existing, ridge_df], ignore_index=True)
        combined.to_csv(ridge_csv, index=False)
    except FileNotFoundError:
        ridge_df.to_csv(ridge_csv, index=False)

    # Run NRI computation ---- fo rf model 
    elas_df = compute_nri_for_survival_models(model_elastic, model_elastic_final, X_val, X_val2, y_val, time_points, risk_thresholds)
    elas_df["percentile"] = percentiles
    elas_df["random_state"] = random_state
    elas_df["model"] = "ELASTIC_NET"
    elas_df["comaprison"] = f"\n🔁 Running comparison: {model1_name} vs {model2_name}"

    # Set file to store cumulative NRI results
    elas_csv =   "./7FEATURE_test.csv"
    # Save or append to CSV
    try:
        existing = pd.read_csv(elas_csv)
        combined = pd.concat([existing, elas_df], ignore_index=True)
        combined.to_csv(elas_csv, index=False)
    except FileNotFoundError:
        elas_df.to_csv(elas_csv, index=False)



🔁 Running comparison: clinical_only vs promoter_only
['Ki67', 'Size_cm']
['pr3004_huwe1', 'pr41492_ftx']
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Best Hyperparameters: {'min_samples_leaf': 5, 'min_samples_split': 2, 'n_estimators': 500}
5
2
500
Best L1 Ratio: 0.2
alpha_lasso =  [0.01367936934223473]
alpha_ridge =  [3016.479783904909]
alpha_elastic =  [0.06528810446868687]
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Best Hyperparameters: {'min_samples_leaf': 5, 'min_samples_split': 2, 'n_estimators': 100}
5
2
100
Best L1 Ratio: 0.2
alpha_lasso =  [0.018420621051909766]
alpha_ridge =  [3701.1319745241544]
alpha_elastic =  [0.10589618957002023]

🔁 Running comparison: clinical_only vs all_features
['Ki67', 'Size_cm']
['Ki67', 'Size_cm', 'pr3004_huwe1', 'pr41492_ftx']
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Best Hyperparameters: {'min_samples_leaf': 5, 'min_samples_split': 2, 'n_estimators': 500}
5
2
500
Best L1 Ratio: 0.2
alpha_las